In [60]:
#### Preamble ####
# Purpose: Cleans and saves the data from IPUMS USA
# Author: Jiazhou(Justin) Bi and Weiyang Li
# Date: 3 October 2024
# Contact: justin.bi@mail.utoronto.ca or weiyang.li@mail.utoronto.ca
# License: MIT
# Pre-requisites: python 3.10.5 or above, with pandas installed for python
# Any other information needed? None

# Loading the data

In [61]:
import pandas as pd

In [62]:
#Loading the dataset as DataFrame
df = pd.read_csv('../data/01-raw_data/raw_data.csv.gz')
print(df.head())

   YEAR  SAMPLE  SERIAL       CBSERIAL  HHWT        CLUSTER  STATEICP  CITY  \
0  2022  202201       1  2022010000031  69.0  2022000000011        41     0   
1  2022  202201       2  2022010000111  22.0  2022000000021        41     0   
2  2022  202201       3  2022010000200  45.0  2022000000031        41     0   
3  2022  202201       4  2022010000261   4.0  2022000000041        41     0   
4  2022  202201       5  2022010000296  47.0  2022000000051        41     0   

   STRATA  GQ  ...  AGE  MARST  EDUC  EDUCD  SCHLTYPE  OCC2010  IND1990  \
0  280301   3  ...   85      5     7     71         1     9920        0   
1  200001   3  ...   51      5     6     64         1     5620      591   
2  280301   3  ...   36      6     2     26         1     8800      100   
3  110001   4  ...   74      6     0      2         1     9920        0   
4  150201   3  ...   49      4     7     71         1     6230       60   

   INCTOT  VETSTAT  VETSTATD  
0   18800        1        11  
1   12500   

In [63]:
#Checking our sample size
print(df.shape)
#(3373378, 26)

(3373378, 26)


# Dropping Columns

In [64]:
df = df.drop(columns=['YEAR','SAMPLE','SERIAL','CBSERIAL','HHWT','CLUSTER','STRATA','OWNERSHP','OWNERSHPD','PERWT','EDUCD','VETSTATD','PERNUM','CITY'])
print(df.head())

   STATEICP  GQ  MORTGAGE  SEX  AGE  MARST  EDUC  SCHLTYPE  OCC2010  IND1990  \
0        41   3         0    2   85      5     7         1     9920        0   
1        41   3         0    1   51      5     6         1     5620      591   
2        41   3         0    2   36      6     2         1     8800      100   
3        41   4         0    1   74      6     0         1     9920        0   
4        41   3         0    1   49      4     7         1     6230       60   

   INCTOT  VETSTAT  
0   18800        1  
1   12500        1  
2   16400        1  
3    8600        1  
4    5000        2  


# Data Validation

In [65]:
missing_values = df.isnull().sum()
print(missing_values)
#PASS. No missing values found in this dataset.

STATEICP    0
GQ          0
MORTGAGE    0
SEX         0
AGE         0
MARST       0
EDUC        0
SCHLTYPE    0
OCC2010     0
IND1990     0
INCTOT      0
VETSTAT     0
dtype: int64


# Changing to Correct Data Types

In [66]:
df.dtypes
# STATEICP    object
# CITY        object
# GQ          object
# OWNERSHP    object
# MORTGAGE    object
# SEX         object
# AGE          int64
# MARST       object
# EDUC        object
# SCHLTYPE    object
# OCC2010     object
# IND1990     int64
# INCTOT       int64
# VETSTAT     object
# dtype: object

STATEICP    int64
GQ          int64
MORTGAGE    int64
SEX         int64
AGE         int64
MARST       int64
EDUC        int64
SCHLTYPE    int64
OCC2010     int64
IND1990     int64
INCTOT      int64
VETSTAT     int64
dtype: object

In [67]:
df['STATEICP']=df['STATEICP'].astype('str')
#The below variable was dropped because it has a lot of missing values.
#df['CITY']=df['CITY'].astype('str')
df['GQ']=df['GQ'].astype('str')
#df['OWNERSHP']=df['OWNERSHP'].astype('str')
df['MORTGAGE']=df['MORTGAGE'].astype('str')
df['SEX']=df['SEX'].astype('str')
df['MARST']=df['MARST'].astype('str')
df['EDUC']=df['EDUC'].astype('str')
df['SCHLTYPE']=df['SCHLTYPE'].astype('str')
df['OCC2010']=df['OCC2010'].astype('str')
df['IND1990']=df['IND1990'].astype('str')
df['VETSTAT']=df['VETSTAT'].astype('str')
df['INCTOT']=df['INCTOT'].astype('str')

# Dropping Income that does not reflect actual income numbers

In [68]:
# INCTOT Specific Variable Codes
# -009995 = -$9,900 (1980)
# -000001 = Net loss (1950)
# 0000000 = None
# 0000001 = $1 or break even (2000, 2005-onward ACS and PRCS)
# 9999999 = N/A
# 9999998 = Unknown
# We will drop all the values above.
df = df[~df['INCTOT'].isin(['-009995', '-000001', '0000000', '0000001', '9999999', '9999998'])]
df['INCTOT']=df['INCTOT'].astype('int')
df.shape
#(2850502, 13)

(2850502, 12)

# Dropping Rows with Not Identified/Missing Value

In [69]:
df = df[(df['STATEICP'] != '99') & (df['MORTGAGE'] != '0') 
        & (df['SEX'] != '9') & (df['AGE'] != '999') & (df['MARST'] != '9') 
        & (df['EDUC'] != '99') & (df['SCHLTYPE'] != '0') & (df['VETSTAT'] != '9') 
        & (df['IND1990'] != '0') & (df['IND1990'] != '999')] 


In [70]:
#Checking our sample size now
print(df.shape)
#(1431351, 13)

(1431351, 12)


# Regroup the Education Levels （EDUC）

In [71]:
# Convert it to integers
df['EDUC'] = df['EDUC'].astype(int)

# Mapping the education levels to the new categories
education_labels = {
    0: 'No Schooling',      # No schooling
    1: 'Nursery School',    # Nursery school to grade 4
    2: 'Primary School',    # Grade 5-8
    3: 'Middle School',     # Grade 9
    4: 'Middle School',     # Grade 10
    5: 'Middle School',     # Grade 11
    6: 'Middle School',     # Grade 12
    7: 'College 1-4 Years', # 1 year of college
    8: 'College 1-4 Years', # 2 years of college
    9: 'College 1-4 Years', # 3 years of college
    10: 'College 1-4 Years', # 4 years of college
    11: 'College 5+ Years', # 5+ years of college
}

# Apply the mapping to the 'EDUC' column in the dataset
df['EDUC_GROUPED'] = df['EDUC'].map(education_labels)

# Display the first few rows of the modified dataframe
df[['EDUC', 'EDUC_GROUPED']].head()


,EDUC,EDUC_GROUPED
3136,10,College 1-4 Years
3144,8,College 1-4 Years
3148,7,College 1-4 Years
3149,7,College 1-4 Years
3150,5,Middle School


In [72]:
# Define the numeric mapping for the new education groups
education_numeric_mapping = {
    'No Schooling': 0,
    'Nursery School': 1,
    'Primary School': 2,
    'Middle School': 3,
    'College 1-4 Years': 4,
    'College 5+ Years': 5
}

# Map the grouped education categories to numeric codes
df['EDUC_new'] = df['EDUC_GROUPED'].map(education_numeric_mapping)

# Display the first few rows to verify
df[['EDUC', 'EDUC_GROUPED', 'EDUC_new']].head()

,EDUC,EDUC_GROUPED,EDUC_new
3136,10,College 1-4 Years,4
3144,8,College 1-4 Years,4
3148,7,College 1-4 Years,4
3149,7,College 1-4 Years,4
3150,5,Middle School,3


In [73]:
# Convert it to objects
df['EDUC_new'] = df['EDUC_new'].astype(object)

df = df.drop(columns=['EDUC_GROUPED'])
print(df.head())

df.dtypes

     STATEICP GQ MORTGAGE SEX  AGE MARST  EDUC SCHLTYPE OCC2010 IND1990  \
3136       41  1        3   2   51     6    10        1     800     850   
3144       41  1        3   1   61     1     8        1    4810     711   
3148       41  1        1   1   63     1     7        1    8030      50   
3149       41  1        1   2   36     4     7        1     120     702   
3150       41  1        1   1   17     6     5        3    4000     641   

      INCTOT VETSTAT EDUC_new  
3136  120000       1        4  
3144  200000       1        4  
3148   40000       1        4  
3149   50000       1        4  
3150     400       1        3  


STATEICP    object
GQ          object
MORTGAGE    object
SEX         object
AGE          int64
MARST       object
EDUC         int64
SCHLTYPE    object
OCC2010     object
IND1990     object
INCTOT       int64
VETSTAT     object
EDUC_new    object
dtype: object

# Saving the DataFrame as a parquet file

In [74]:
df.to_parquet('../data/02-analysis_data/cleaned_data.parquet', index=False)